# Constitution d'une base adresse à partir de fichiers DVF
- Adresse complète
- Code commune INSEE
- Code IRIS (code quartier de l'INSEE)
- Coordonnées GPS de la commune
- Coorodnnées GPS de l'adresse

In [1]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

In [3]:
year = '2019'

In [4]:
import logging
logging.basicConfig(filename='./export_gps_iris/get_gps_iris_' + year + '.log', level=logging.INFO)

with open('get_gps_iris.log', 'w'):
    pass

## Lecture du fichier Adresses

In [5]:
# Chargement du jeu de données
df = pd.read_csv('01A - export_dvf_adresses_' + year + '.csv', sep=",",
                dtype = {'Code postal': str, 'Code departement': str, 'Code commune INSEE': str})
df.head()

,Unnamed: 0,Unnamed: 0.1,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse
0,2,3,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY
1,3,4,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS
2,4,5,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT
3,5,6,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE
4,6,7,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE


In [6]:
print(df.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'No voie', 'Type de voie', 'Voie',
       'Code postal', 'Commune', 'Code departement', 'Code commune INSEE',
       'Ville', 'Adresse'],
      dtype='object')


## Paramètres

In [7]:
# Renseigner ici la première ligne et la dernière ligne de la base adresses qui vont être utilisées
row_first = 0
row_last = df.shape[0]

### Renseigner manuellement si on souhaite ne parser qu'une partie des adresses
row_first = 0
row_last = 2500

print("Calcul des coordonnées GPS et Codes IRIS de " + str(row_first) + " à " + str(row_last - 1))
logging.info("Calcul des coordonnées GPS et Codes IRIS de " + str(row_first) + " à " + str(row_last - 1))

Calcul des coordonnées GPS et Codes IRIS de 0 à 2499


In [8]:
# Fréquence de l'export intermédiaire (si problème d'API)
freq_export = 1000

## Sélection des adresses à parser pour ajout des coordonnées GPS et codes IRIS

In [9]:
# Sélection des adresses à partir de row_first et row_last définies plus haut
selec = df.iloc[row_first:row_last, :]
selec.shape

(2500, 11)

In [10]:
# Instanciation de la BDD de correspondance entre Code Postal / Communes et Code INSEE + GPS Communes
# La BDD se trouve ici : https://data.opendatasoft.com/explore/dataset/code-postal-code-insee-2015@public/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B
communes = pd.read_csv('databases/code-postal-code-insee-2015.csv', sep = ";")
communes.head()

,Geo Point,Geo Shape,CODE_COM,INSEE_COM,NOM_COM,STATUT,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,POPULATION,CODE_CANT,CODE_ARR,CODE_DEPT,NOM_DEPT,Code_postal,Nom_commune,coordonnees_gps,Unnamed: 18
0,"49.0097923668,0.696073334129","{""type"": ""Polygon"", ""coordinates"": [[[0.682900...",49,27049,BEAUMESNIL,Commune simple,531927,6881409,162,1264,562,2.0,2.0,27,EURE,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297",NaN
1,"49.0521199676,0.412267826158","{""type"": ""Polygon"", ""coordinates"": [[[0.409543...",547,27547,SAINT-GERMAIN-LA-CAMPAGNE,Commune simple,510734,6886673,176,2231,895,3.0,2.0,27,EURE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157",NaN
2,"48.7896521776,0.276557382012","{""type"": ""Polygon"", ""coordinates"": [[[0.260047...",347,61347,RESENLIEU,Commune simple,500194,6858135,231,504,210,21.0,2.0,61,ORNE,61230.0,RESENLIEU,"48.7900887821,0.276930426076",NaN
3,"48.8470682746,0.643176111978","{""type"": ""Polygon"", ""coordinates"": [[[0.628763...",359,27359,JUIGNETTES,Commune simple,527172,6863726,228,1302,243,6.0,3.0,27,EURE,27250.0,JUIGNETTES,"48.8472706788,0.64345379282",NaN
4,"48.8662500091,0.559682765078","{""type"": ""Polygon"", ""coordinates"": [[[0.553791...",136,61136,COUVAINS,Commune simple,521103,6865598,236,1788,167,18.0,2.0,61,ORNE,NaN,NaN,NaN,NaN


In [11]:
# On ne garde que les colonnes qui nous intéressent 
# (On a les coordonnees gps dedans, ce sera intéressant pour faire un check des coordonnees qu'on récupère plus bas)

communes = communes[['INSEE_COM', 'NOM_COM', 'Code_postal', 'Nom_commune', 'coordonnees_gps']]
communes = communes.rename(columns = {'NOM_COM': "Commune"})
communes.head()

,INSEE_COM,Commune,Code_postal,Nom_commune,coordonnees_gps
0,27049,BEAUMESNIL,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297"
1,27547,SAINT-GERMAIN-LA-CAMPAGNE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157"
2,61347,RESENLIEU,61230.0,RESENLIEU,"48.7900887821,0.276930426076"
3,27359,JUIGNETTES,27250.0,JUIGNETTES,"48.8472706788,0.64345379282"
4,61136,COUVAINS,NaN,NaN,NaN


In [12]:
# Instanciation d'un dataframe communes_ville qui ne conserve que le code INSEE, les coordonnés, et la ville
communes_ville = communes
communes_ville['Code_postal'] = communes_ville['Code_postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
communes_ville['Ville'] = communes_ville['Code_postal'] + " " + communes_ville['Commune']
communes_ville['Ville'] = communes_ville['Ville'].where(-communes_ville['Code_postal'].isna(), communes_ville['Commune'])
communes_ville = communes_ville.drop(columns = {'Commune', 'Code_postal', 'Nom_commune'})
communes_ville['Ville'] = communes_ville['Ville'].replace('-', ' ', regex=True).astype(str)
communes_ville.head()

,INSEE_COM,coordonnees_gps,Ville
0,27049,"49.0090520551,0.696415474297",27330 BEAUMESNIL
1,27547,"49.0522921876,0.412852228157",27230 SAINT GERMAIN LA CAMPAGNE
2,61347,"48.7900887821,0.276930426076",61230 RESENLIEU
3,27359,"48.8472706788,0.64345379282",27250 JUIGNETTES
4,61136,NaN,COUVAINS


In [13]:
# Ajout du code commune INSEE et Coordonnées GPS à la sélection d'adresses
selec = selec.merge(communes_ville, on = "Ville", how = "left")
selec = selec.drop(columns = {'Unnamed: 0'})
selec.head()

,Unnamed: 0.1,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,INSEE_COM,coordonnees_gps
0,3,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,01314,"46.0197521486,5.28523353803"
1,4,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,01350,"46.2749447261,5.28428557632"
2,5,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,01024,"46.2861802203,5.1795233845"
3,6,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,01106,"46.2003848172,5.45082526739"
4,7,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,01106,"46.2003848172,5.45082526739"


In [14]:
selec['Commune_Lat'] = selec['coordonnees_gps'].str.split(",", expand = True)[0]
selec['Commune_Lon'] = selec['coordonnees_gps'].str.split(",", expand = True)[1]
selec = selec.drop(columns = {'coordonnees_gps'})
selec.head(15)

,Unnamed: 0.1,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon
0,3,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,01314,46.0197521486,5.28523353803
1,4,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,01350,46.2749447261,5.28428557632
2,5,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,01024,46.2861802203,5.1795233845
3,6,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,01106,46.2003848172,5.45082526739
4,7,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,01106,46.2003848172,5.45082526739
5,8,50.0,RUE,DOC NODET,1000,BOURG-EN-BRESSE,01,01053,1000 BOURG EN BRESSE,50 RUE DOC NODET 1000 BOURG-EN-BRESSE,01053,46.2051520382,5.24602125501
6,11,NaN,NaN,EN CHEVRET,1250,TOSSIAT,01,01422,1250 TOSSIAT,EN CHEVRET 1250 TOSSIAT,01422,46.1384790158,5.30400055922
7,13,1724.0,RTE,DE MONTLEGER,1560,MANTENAY-MONTLIN,01,01230,1560 MANTENAY MONTLIN,1724 RTE DE MONTLEGER 1560 MANTENAY-MONTLIN,01230,46.4302754511,5.10675899732
8,16,1250.0,RTE,DE CUET,1340,MONTREVEL-EN-BRESSE,01,01266,1340 MONTREVEL EN BRESSE,1250 RTE DE CUET 1340 MONTREVEL-EN-BRESSE,01266,46.3252811442,5.11295966315
9,19,181.0,RTE,DE VERNOUX,1560,COURTES,01,01128,1560 COURTES,181 RTE DE VERNOUX 1560 COURTES,01128,46.4570082032,5.11128546723


In [15]:
selec['code_iris'] = ""
selec['lon'] = ""
selec['lat'] = ""
selec.head()

,Unnamed: 0.1,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,3,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,01314,46.0197521486,5.28523353803,,,
1,4,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,01350,46.2749447261,5.28428557632,,,
2,5,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,01024,46.2861802203,5.1795233845,,,
3,6,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,01106,46.2003848172,5.45082526739,,,
4,7,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,01106,46.2003848172,5.45082526739,,,


In [16]:
selec.shape

(2500, 16)

## Récupération des codes IRIS et coordonnées GPS à partir de l'API Nominatim et geo.api.gouv

In [17]:
# Get GPS Coordinates + IRIS Code from addresses

geolocator = Nominatim(user_agent="pyPredImmo")

start = time.time()
etape = time.time()
print("Heure début : ")
print(time.strftime('%H:%M:%S', time.gmtime(start)))
print("\n")

logging.info("Heure début : " + time.strftime('%H:%M:%S', time.gmtime(start)) + "\n")

n=0

for row in range(0, row_last - row_first):
    
    iris = ""
    longitude_api = ""
    latitude_api = ""
    longitude_adresse = ""
    latitude_adresse = ""

    
    adresse = selec['Adresse'][row]
    ville = selec['Ville'][row]
    code_INSEE = selec['Code commune INSEE'][row]

    # Interrogation de l'API Nominatim
    try:
        location = geolocator.geocode(adresse, country_codes = 'fr', timeout = 5)
    
    except requests.exceptions.RequestException as e:
        latitude_api = ""
        longitude_api = ""
        raise SystemExit(e)
        logging.info(adresse + ": " + e)
        
    if location is None:                      # Sinon aucun résultat, on ne met rien dans les coordonnées GPS qui vont servir à interroger les codes IRIS
        latitude_api = ""
        longitude_api = ""
    
    else:
        latitude_api = location.latitude
        longitude_api = location.longitude
        
        if isinstance(code_INSEE, float):     # Si pas de code commune
            print(adresse + ': pas de code commune INSEE')
            logging.info(adresse + ': pas de code commune INSEE')
            iris = ""
        elif latitude_api == "":               # Si pas de coordonnées GPS
            print(adresse + ": pas de coordonnees GPS pour code IRIS")
            logging.info(adresse + ": pas de coordonnees GPS pour code IRIS")
            iris = ""
        else:                              # Sinon on interroge l'API du gouv pour récupérer le code IRIS
            try:
                result = requests.get('https://geo.api.gouv.fr/iris?lon=' + str(longitude_api) + '&lat=' 
                                 + str(latitude_api) + '&codeCommune=' + str(code_INSEE), timeout = 5)
            except requests.exceptions.RequestException as e: 
                logging.info(adresse + ": " + e)
                raise SystemExit(e)
            
            if result.ok == True:
                obj = json.loads(result.text)
                iris = obj['codeIris']
                #print(obj['codeIris'])
            else:
                result.close()
            print(adresse + " ===> " + str(latitude_api) + "," + str(longitude_api) + " (code IRIS = " + str(iris) + ")")
   
        
    selec['code_iris'][row] = iris
    selec['lon'][row] = longitude_api
    selec['lat'][row] = latitude_api
    
    n = n + 1
    
    if (n % 100 == 0):
        print("\n")
        print("Etape : " + str(n))
        print(time.strftime('%H:%M:%S', time.gmtime(time.time())))
        print("Temps pour 100 : " + str(time.time() - etape) + " sec")
        print("\n")
        logging.info("Etape : " + str(n) + ", Temps pour 100 items: " + str(time.time() - etape) + " sec")
        etape = time.time()

    if (n % freq_export == 0):
        selec.iloc[(n - freq_export) : n, :].to_csv(
            './export_gps_iris/export_adresses_inter_' + str(row_first + n - freq_export) + "-" + str(row_first + n - 1) + '_' + year + '.csv')
        logging.info("02 - Export intermédiaire : " + str(row_first + n - freq_export) + "-" + str(row_first + n - 1))

        
end = time.time()
print("\n")
print("Heure fin : ")
print(time.strftime('%H:%M:%S', time.gmtime(end)))
print("\n")

logging.info("Heure fin : " + time.strftime('%H:%M:%S', time.gmtime(end)) + "\n")
logging.info("Temps de calcul : " + str(end - start) + ' sec')

print("Temps de calcul : ")
print(str(end - start) + ' sec')

Heure début : 
11:20:31


3 RUE DES CHAMPAGNES 1160 PRIAY ===> 46.0019301,5.2876139 (code IRIS = 013140000)
165 ALL DES LIBELLULES 1340 ATTIGNAT ===> 46.2690381,5.1745553 (code IRIS = 010240000)
9 RTE DU VIADUC 1250 CIZE ===> 46.205148,5.452365 (code IRIS = 011060000)
SUR LA LATIE 1250 CIZE ===> 46.2054765,5.4531452 (code IRIS = 011060000)
1724 RTE DE MONTLEGER 1560 MANTENAY-MONTLIN ===> 46.437995,5.121439 (code IRIS = 012300000)
1250 RTE DE CUET 1340 MONTREVEL-EN-BRESSE ===> 46.321708,5.112519 (code IRIS = 012660000)
181 RTE DE VERNOUX 1560 COURTES ===> 46.468146,5.103709 (code IRIS = 011280000)
CROCU 1560 SAINT-TRIVIER-DE-COURTES ===> 46.4628038,5.0427754 (code IRIS = 013880000)
LE PIOCHAY 1560 SAINT-TRIVIER-DE-COURTES ===> 46.4691123,5.0883614 (code IRIS = 013880000)
GRANDVAL 1560 SAINT-TRIVIER-DE-COURTES ===> 46.4606544,5.0557583 (code IRIS = 013880000)
5575  MOLARDOURY 1560 SAINT-TRIVIER-DE-COURTES ===> 46.4568175,5.0653117 (code IRIS = 013880000)
5362  LA MORANDIERE 1240 CERTINES

28 RUE DE FRANCHE COMTE 1250 NIVIGNE ET SURAN ===> 46.2670214,5.4265621 (code IRIS = 010950000)
10 RUE BRICHEMER 1000 BOURG-EN-BRESSE ===> 46.2127046,5.2522479 (code IRIS = 010530502)
CHALANDRE 1000 SAINT-DENIS-LES-BOURG ===> 46.20257875,5.189708614071642 (code IRIS = 013440000)
4 IMP DES HIRONDELLES 1190 PONT-DE-VAUX ===> 46.43283,4.94236 (code IRIS = 013050000)
6 RUE LOUIS BLERIOT 1000 BOURG-EN-BRESSE ===> 46.2116055,5.2356948 (code IRIS = 010530603)
5386  LA POTIERE 1340 MONTREVEL-EN-BRESSE ===> 46.3087372,5.0991369 (code IRIS = 012660000)
LA POTIERE 1340 MONTREVEL-EN-BRESSE ===> 46.3087372,5.0991369 (code IRIS = 012660000)
49 ALL DE LA GARE 1340 ATTIGNAT ===> 46.287456,5.1584449 (code IRIS = 010240000)
471 RUE DES ADAMS 1340 CRAS-SUR-REYSSOUZE ===> 46.3130788,5.1631944 (code IRIS = 011300000)
LES ADAMS 1340 CRAS-SUR-REYSSOUZE ===> 46.3134688,5.1637043 (code IRIS = 011300000)
150 AV FRANCOIS PIGNIER 1000 BOURG-EN-BRESSE ===> 46.2185892,5.2454473 (code IRIS = 010530701)
24 BD JULES F

CHE DU BOURG 1000 SAINT-DENIS-LES-BOURG ===> 46.2020182,5.1886769 (code IRIS = 013440000)


Etape : 300
11:23:06
Temps pour 100 : 51.25305938720703 sec


40 RUE DU VILLAGE 1000 SAINT-DENIS-LES-BOURG ===> 46.2006366,5.1898194 (code IRIS = 013440000)
261 RTE DE GRIEGES 1290 CORMORANCHE-SUR-SAONE ===> 46.2352532,4.8447423 (code IRIS = 011230000)
738 CHE DES VIGNES 1290 CORMORANCHE-SUR-SAONE ===> 46.2352943,4.8439074 (code IRIS = 011230000)
11 RUE HENRI DUNANT 1000 BOURG-EN-BRESSE ===> 46.1804664,5.3208671 (code IRIS = )
461 RUE DU DOCTEUR HUBERT 1160 NEUVILLE-SUR-AIN ===> 46.0782372,5.376654 (code IRIS = 012730000)
16 RUE LAMARTINE 1000 BOURG-EN-BRESSE ===> 46.2028099,5.2041821 (code IRIS = 010530202)
237 RUE BRICHEMER 1000 BOURG-EN-BRESSE ===> 46.213313,5.251875 (code IRIS = 010530502)
2 RUE DU PALAIS 1000 BOURG-EN-BRESSE ===> 46.2047895,5.2239094 (code IRIS = 010530101)
LES RODETS 1270 VILLEMOTIER ===> 46.335371,5.3170981 (code IRIS = 014450000)
395 RTE DE STRASBOURG 1270 VILLEMOTIER ==

521 ALL DU THIOUDET 1960 PERONNAS ===> 46.1706942,5.1945378 (code IRIS = 012890102)
20 ALL DE LA PETITE REYSSOUZE 1000 BOURG-EN-BRESSE ===> 46.2014126,5.2370937 (code IRIS = 010530501)
18 ALL DE LA PETITE REYSSOUZE 1000 BOURG-EN-BRESSE ===> 46.2014094,5.2370826 (code IRIS = 010530501)
LES FOURNILS 1250 JASSERON ===> 46.2092594,5.3235025 (code IRIS = 011950000)
193 CHE DU GOZ 1250 JASSERON ===> 46.2099266,5.3224498 (code IRIS = 011950000)
VILLAGE 1250 JASSERON ===> 46.2122209,5.3255283 (code IRIS = 011950000)
233 CHE DES SAULES 1960 PERONNAS ===> 46.1844987,5.2011945 (code IRIS = 012890101)
5167  OUSSIAT 1160 PONT-D AIN ===> 46.0557947,5.3611957 (code IRIS = 013040000)
5163  OUSSIAT 1160 PONT-D AIN ===> 46.0557947,5.3611957 (code IRIS = 013040000)
LES BELOUSES 1380 BAGE-LE-CHATEL ===> 46.3116324,4.9355573 (code IRIS = 010260000)
LA PLAINE 1250 JASSERON ===> 46.2139808,5.3128771 (code IRIS = 011950000)
53 RUE LOUISE DE SAVOIE 1160 PONT-D AIN ===> 46.050662,5.3437344 (code IRIS = 01304000

2880 RTE DE PONT DE VAUX 1560 SAINT-TRIVIER-DE-COURTES ===> 46.452841,5.040421 (code IRIS = 013880000)
SUR LA ROUTE 1340 ATTIGNAT ===> 46.289384,5.16355 (code IRIS = 010240000)
LA PAYSANNE 1160 SAINT-MARTIN-DU-MONT ===> 46.1051705,5.3271277 (code IRIS = 013740000)
5450  LE BOURG 1340 MALAFRETAZ ===> 46.3218,5.14516 (code IRIS = 012290000)
EN PLAMONT 1160 SAINT-MARTIN-DU-MONT ===> 46.0988272,5.3493056 (code IRIS = 013740000)
LES ROCHETTES 1160 SAINT-MARTIN-DU-MONT ===> 46.0971195,5.3340437 (code IRIS = 013740000)
SOUS COULIS 1160 SAINT-MARTIN-DU-MONT ===> 46.098938,5.3468301 (code IRIS = 013740000)


Etape : 600
11:25:38
Temps pour 100 : 51.116400718688965 sec


LES COULIS 1160 SAINT-MARTIN-DU-MONT ===> 46.098938,5.3468301 (code IRIS = 013740000)
10 RUE NOTRE DAME DES CHAMPS 1190 PONT-DE-VAUX ===> 46.43545,4.936757 (code IRIS = 013050000)
LE CANTON 1250 JASSERON ===> 46.2317904,5.2917693 (code IRIS = 011950000)
1 RUE SAINT EXUPERY 1160 PONT-D AIN ===> 46.0493333,5.3358711 (code IRIS = 0

...

4 RUE DES GABONNIERES 1290 PONT-DE-VEYLE ===> 46.2623723,4.8855005 (code IRIS = 013060000)
824 RTE DE PONT DE VEYLE 1290 GRIEGES ===> 46.26261,4.859683 (code IRIS = 011790000)
70 RTE DES COURBETS 1380 BAGE-DOMMARTIN ===> 46.334258,4.938094 (code IRIS = 010250000)
5799  PATARD 1340 MONTREVEL-EN-BRESSE ===> 46.3232936,5.1083576 (code IRIS = 012660000)
1925 RTE DE MACON 1310 POLLIAT ===> 46.2501663,5.1180678 (code IRIS = 013010000)
CHERINAL 1310 POLLIAT ===> 46.2546975,5.1008902 (code IRIS = 013010000)
CORGENT 1340 JAYAT ===> 46.3715453,5.0972341 (code IRIS = 011960000)
74 RTE DE CORGENT 1340 JAYAT ===> 46.3693997,5.0961859 (code IRIS = 011960000)
12 RUE CHARLES ROBIN 1000 BOURG-EN-BRESSE ===> 46.2056221,5.2298289 (code IRIS = 010530102)
7 RUE JEAN BAPTISTE CLEMENT 1000 BOURG-EN-BRESSE ===> 46.192051,5.2091799 (code IRIS = 010530203)
RUE JEAN MERMOZ 1000 BOURG-EN-BRESSE ===> 46.1918032,5.2096945 (code IRIS = 010530203)
8 RUE PASTEUR 1000 BOURG-EN-BRESSE ===> 46.2056885,5.2262092 (code IRI

1820 RTE DE CRANGEAT 1440 VIRIAT ===> 46.2695488,5.2103711 (code IRIS = 014510101)
10 ALL DES CHAMPS 1310 MONTRACOL ===> 46.1918453,5.1336445 (code IRIS = 012640000)
242 ALL DU SORBIER 1340 ATTIGNAT ===> 46.2791134,5.1605393 (code IRIS = 010240000)


Etape : 2000
11:37:32
Temps pour 100 : 51.15731692314148 sec


MAISONS ROUGES 1250 JASSERON ===> 46.2019747,5.3245825 (code IRIS = 011950000)
PRE GUERIN 1340 MARSONNAS ===> 46.3348005,5.0942561 (code IRIS = 012360000)
806 RTE DE MARBOZ 1440 VIRIAT ===> 46.2348182,5.229509 (code IRIS = 014510101)
FRAIDEGUE 1370 BENY ===> 46.3036652,5.2634579 (code IRIS = 010380000)
3180 RTE DES BLANCS 1370 BENY ===> 46.3151425,5.2829266 (code IRIS = 010380000)
5175  LE GOMET 1270 PIRAJOUX ===> 46.3708779,5.2916715 (code IRIS = 012960000)
LE VILLARD 1270 DOMSURE ===> 46.4011057,5.2899894 (code IRIS = 011470000)
75 IMP DE LA MITTONIERE 1270 DOMSURE ===> 46.401764,5.287223 (code IRIS = 011470000)
LA MITTONIERE 1270 DOMSURE ===> 46.401423,5.2871839 (code IRIS =

5144  TRECONNAS 1250 CEYZERIAT ===> 46.1863403,5.3172491 (code IRIS = 010720000)
1352 AV DE LYON 1960 PERONNAS ===> 46.1899509,5.2109866 (code IRIS = 012890101)
5040  A L'HORLOGE 1380 BAGE-LE-CHATEL ===> 46.3054556,4.9285869 (code IRIS = 010260000)
16  LE GRAND FAUBOURG 1190 PONT-DE-VAUX ===> 46.4386184,4.9414498 (code IRIS = 013050000)
18  LE GRAND FAUBOURG 1190 PONT-DE-VAUX ===> 46.4386184,4.9414498 (code IRIS = 013050000)
LA BRIGONNIERE 1851 MARBOZ ===> 46.3564901,5.2313979 (code IRIS = 012320000)
5754  LA VALLEE 1190 PONT-DE-VAUX ===> 46.4269057,4.9515856 (code IRIS = 013050000)
LA VALLEE 1190 PONT-DE-VAUX ===> 46.4269057,4.9515856 (code IRIS = 013050000)
294 RUE DE LA SOZAYE 1570 MANZIAT ===> 46.364998,4.904077 (code IRIS = 012310000)
1 ALL DU VIEUX CHENE 1310 SAINT-MARTIN-LE-CHATEL ===> 46.278382,5.114088 (code IRIS = 013750000)
5505  MACHARD 1340 MONTREVEL-EN-BRESSE ===> 46.3336905,5.1056989 (code IRIS = 012660000)
MACHARD 1340 MONTREVEL-EN-BRESSE ===> 46.3336905,5.1056989 (code

24 CHE DES PONTHUS 1340 BRESSE VALLONS ===> 46.3190739,5.1602792 (code IRIS = 011300000)
AU MOTTIER 1750 REPLONGES ===> 46.3076518,4.8878263 (code IRIS = 013200000)
5348  LES CAROUGES 1340 MONTREVEL-EN-BRESSE ===> 46.3201122,5.1113932 (code IRIS = 012660000)
263 ALL DE L'AULNAIE 1340 ATTIGNAT ===> 46.2823221,5.1596946 (code IRIS = 010240000)
191 CHE DE L'ALAGNIER 1000 BOURG-EN-BRESSE ===> 46.1963909,5.259099 (code IRIS = 010530701)
100 RUE SAMUEL GUICHENON 1250 CEYZERIAT ===> 46.1793864,5.31505 (code IRIS = 010720000)
1008 RTE DES BOUTONNIERS 1190 CHAVANNES-SUR-REYSSOUZE ===> 46.4326126,5.0215013 (code IRIS = 010940000)
397 RUE DE LA LEVEE 1750 REPLONGES ===> 46.3042105,4.8456099 (code IRIS = 013200000)
3 PL BELLECOUR 1290 PONT-DE-VEYLE ===> 46.2613389,4.8872518 (code IRIS = 013060000)
52 RTE DE CIZE 1250 HAUTECOURT-ROMANECHE ===> 46.159959,5.4174726 (code IRIS = 011840000)
LES BOURBOUILLONS 1250 DROM ===> 46.2158698,5.3644393 (code IRIS = 011500000)
1338 RTE DU VILLAGE 1250 MONTAGNAT 

5069 CHE DES ROLIETTES 1160 PRIAY ===> 46.0267627,5.2965827 (code IRIS = 013140000)
210 RUE GEORGE SAND 1000 SAINT-DENIS-LES-BOURG ===> 46.210477,5.200036 (code IRIS = 013440000)
79 RUE DE LA LEVEE 1750 SAINT-LAURENT-SUR-SAONE ===> 46.3047785,4.840469 (code IRIS = 013700000)
35 IMP DE L'HOTEL 1160 VARAMBON ===> 46.0398535,5.3164856 (code IRIS = 014300000)
262 CHE DES OURES 1000 SAINT-DENIS-LES-BOURG ===> 46.21374,5.200668 (code IRIS = 013440000)
32 CHE DU GRAND CHENE 1380 BAGE-DOMMARTIN ===> 46.3175768,4.9419598 (code IRIS = 010250000)
59 IMP DES RAFFOURS 1270 VERJON ===> 46.3502861,5.3506368 (code IRIS = 014320000)
77 ALL DES DOMBES 1960 PERONNAS ===> 46.1766803,5.2060362 (code IRIS = 012890101)
269 RUE GRANDE RUE 1340 BEREZIAT ===> 46.3655561,5.0459766 (code IRIS = 010400000)
234 RTE DE BOURG 1560 SAINT-TRIVIER-DE-COURTES ===> 46.457385,5.083456 (code IRIS = 013880000)
229 CHE DE LA RICHONNIERE 1000 SAINT-DENIS-LES-BOURG ===> 46.2273665,5.173652 (code IRIS = 013440000)
6046 RUE SAINT

In [18]:
selec.head()

,Unnamed: 0.1,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,3,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,01314,46.0197521486,5.28523353803,013140000,5.287614,46.00193
1,4,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,01350,46.2749447261,5.28428557632,,,
2,5,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,01024,46.2861802203,5.1795233845,010240000,5.174555,46.269038
3,6,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,01106,46.2003848172,5.45082526739,011060000,5.452365,46.205148
4,7,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,01106,46.2003848172,5.45082526739,011060000,5.453145,46.205477


In [19]:
selec.shape

(2500, 16)

## Export de la totalité de la sélection d'adresses après la fin du job

In [20]:
selec.to_csv('./export_gps_iris/02 - export_adresses_' + str(row_first) + "-" + str(row_last - 1) + '_' + year + '.csv', index = False)